In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer

import sklearn

from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.svm import SVC

from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB, ComplementNB, BernoulliNB, CategoricalNB

from IPython.display import clear_output

import pandas as pd
import numpy as np
import time
import string
from tqdm import notebook
from tqdm.auto import tqdm

import warnings
warnings.filterwarnings("ignore")

# Helpers

In [2]:
def progress(done, left, start):
    # Print bar
    clear_output(wait=True)
    print(f"[{'#' * done}{' ' * left}] {round(done/(done+left) * 100, 2)}% ({done}/{done+left})")
    
    # Bereken gepasseerde tijd
    current = time.time()
    hours = int((current - start) // 3600)
    minutes = int((current - start) % 3600 // 60)
    seconds = int((current - start) % 60)
    
    # Print tijd
    print(f"""Time passed = {hours:02d}:{minutes:02d}:{seconds:02d}""")
    
    new_done = done + 1
    new_left = left - 1
    
    return new_done, new_left


# Laad data in

In [3]:
equal = pd.read_csv("../data/training_videos.csv").drop("Unnamed: 0", axis = 1)
equal["full_text"].fillna("", inplace=True)

equal.head()

,channel,video_id,conspiracy,full_text
0,UCyuAKnN3g2fZ7_R9irgEUZQ,SECdW8n4ZGw,False,abraham lincoln came power abraham lincoln com...
1,UCAtdV6VWa02UpCB2YHuVmHg,jSXIrZx8lI4,False,thanksgiv celebr announc union rescu mission n...
2,UCASQzDb4SUC0gZNPbwBubIQ,s0YjJVyyVjI,False,unc health s mobil clinic aim reduc racial eth...
3,UC_giJ3xlEL9jUF1YfJdzzuQ,VMHCvdRdq10,False,establish impur accept criteria part spec dmf ...
4,UCT8RMFbTJV5ILaVykrluOQg,KKiR4FWOZ4w,False,ten panda cub make public debut ahead spring f...


# Vectorize

In [4]:
# Vectorize words
v = TfidfVectorizer(max_df=.75, min_df=2)
x = v.fit_transform(equal['full_text'])

x

<44312x140970 sparse matrix of type '<class 'numpy.float64'>'
	with 23152699 stored elements in Compressed Sparse Row format>

In [5]:
x.shape

(44312, 140970)

In [6]:
X = x
Y = equal['conspiracy'].values.ravel()

# 10% of the data will be the test set
X_trainval, X_test, y_trainval, y_test = sklearn.model_selection.train_test_split(X, Y, test_size=0.1,\
                                                                                  random_state=0)

# of the remaining 90%, 90% will be training set
X_train, X_valid, y_train, y_valid = sklearn.model_selection.train_test_split(X_trainval, y_trainval,\
                                                                              train_size=0.9,\
                                                                              random_state=0)

# K-nearest neighbors

## Hyperparameter tuning

In [7]:
measures = {"K":[], "Accuracy":[], "Precision":[],"Recall":[], "F1":[]}

for k in notebook.tqdm(range(1, 16)):
    knn = KNeighborsClassifier(k).fit(X_train, y_train)

    y_pred = knn.predict(X_valid)

    measures["K"].append(k)
    measures["Accuracy"].append(sklearn.metrics.accuracy_score(y_valid, y_pred))
    measures["Precision"].append(sklearn.metrics.precision_score(y_valid, y_pred))
    measures["Recall"].append(sklearn.metrics.recall_score(y_valid, y_pred))
    measures["F1"].append(sklearn.metrics.f1_score(y_valid, y_pred))

df_knn = pd.DataFrame(measures)
df_knn #.to_csv("k-nearest_neighbors.csv")

  0%|          | 0/15 [00:00<?, ?it/s]

,K,Accuracy,Precision,Recall,F1
0,1,0.889669,0.888456,0.896314,0.892368
1,2,0.872618,0.935043,0.806388,0.865963
2,3,0.888415,0.882212,0.901720,0.891859
3,4,0.879137,0.908899,0.848157,0.877478
4,5,0.873370,0.858482,0.900246,0.878868
5,6,0.873119,0.882441,0.866830,0.874566
6,7,0.868355,0.848891,0.902703,0.874970
7,8,0.867603,0.867382,0.874201,0.870778
8,9,0.861585,0.835672,0.907125,0.869934
9,10,0.859579,0.854397,0.873710,0.863946


## Optimal configuration

In [12]:
knn = KNeighborsClassifier(1).fit(X_train, y_train)

y_pred = knn.predict(X_test)

print("Accuracy :", sklearn.metrics.accuracy_score(y_test, y_pred))
print("Precision:", sklearn.metrics.precision_score(y_test, y_pred))
print("Recall   :", sklearn.metrics.recall_score(y_test, y_pred))
print("F1       :", sklearn.metrics.f1_score(y_test, y_pred))

Accuracy : 0.8786101083032491
Precision: 0.8621752531924263
Recall   : 0.896930829134219
F1       : 0.8792096991468343


# Neural Network

## Hyperparameter tuning

In [9]:
measures = {"Activation function":[], "# of hidden layers":[], "Neurons per layer":[],
            "Accuracy":[], "Precision":[],"Recall":[], "F1":[]}

done = 0
left = 4*3*3
start = time.time()

for act in ['identity', 'logistic', 'tanh', 'relu']:
    for layers in [1, 10, 25]:
        for neurons in [1, 10, 20]:
            # Progress bar
            done, left = progress(done, left, start)
            
            # Fit
            mlp = MLPClassifier(hidden_layer_sizes=[neurons]*layers, activation=act,\
                    random_state=0).fit(X_train, y_train)

            # Predict en sla waardes op
            y_pred = mlp.predict(X_test)
            measures["Activation function"].append(act)
            measures["# of hidden layers"].append(layers)
            measures["Neurons per layer"].append(neurons)
            
            measures["Accuracy"].append(sklearn.metrics.accuracy_score(y_valid, mlp.predict(X_valid)))
            measures["Precision"].append(sklearn.metrics.precision_score(y_valid, mlp.predict(X_valid)))
            measures["Recall"].append(sklearn.metrics.recall_score(y_valid, mlp.predict(X_valid)))
            measures["F1"].append(sklearn.metrics.f1_score(y_valid, mlp.predict(X_valid)))
            
df_nn = pd.DataFrame(measures)
df_nn.to_csv("neural_network.csv")

[################################### ] 97.22% (35/36)
Time passed = 08:495:55


## Optimal configuration

In [8]:
mlp = MLPClassifier(hidden_layer_sizes=[10]*25, activation="identity",
                    random_state=0).fit(X_train, y_train)

y_pred = mlp.predict(X_test)

print("Accuracy :", sklearn.metrics.accuracy_score(y_test, y_pred))
print("Precision:", sklearn.metrics.precision_score(y_test, y_pred))
print("Recall   :", sklearn.metrics.recall_score(y_test, y_pred))
print("F1       :", sklearn.metrics.f1_score(y_test, y_pred))

Accuracy : 0.9135830324909747
Precision: 0.9139834406623735
Recall   : 0.9102153000458085
F1       : 0.91209547854028


# Support-vector machine

## Hyperparameter tuning

In [10]:
measures = {"Kernel":[], "C":[], "Accuracy":[], "Precision":[], "Recall":[], "F1":[]}

done = 0
left = 4*4
start = time.time()

for kernel in ["linear", "poly", "rbf", "sigmoid"]:
    for C in [0.1, 1, 10, 100]:       
        # Print progress
        done, left = progress(done, left, start)
        
        # Train
        svm = SVC(C=C, kernel=kernel, max_iter = 10000).fit(X_train, y_train)
        
        # Predict
        y_pred = svm.predict(X_valid)

        # Store
        measures["Kernel"].append(kernel)
        measures["C"].append(C)

        measures["Accuracy"].append(sklearn.metrics.accuracy_score(y_valid, y_pred))
        measures["Precision"].append(sklearn.metrics.precision_score(y_valid, y_pred))
        measures["Recall"].append(sklearn.metrics.recall_score(y_valid, y_pred))
        measures["F1"].append(sklearn.metrics.f1_score(y_valid, y_pred))
        
df_svm = pd.DataFrame(measures)
df_svm.to_csv("support-vector_machine.csv")

[############### ] 93.75% (15/16)
Time passed = 07:431:58


## Optimal configuration

In [36]:
svm = SVC(C=10, kernel="rbf", max_iter = 10000).fit(X_train, y_train)

y_pred = svm.predict(X_test)

print("Accuracy :", sklearn.metrics.accuracy_score(y_test, y_pred))
print("Precision:", sklearn.metrics.precision_score(y_test, y_pred))
print("Recall   :", sklearn.metrics.recall_score(y_test, y_pred))
print("F1       :", sklearn.metrics.f1_score(y_test, y_pred))

Accuracy : 0.921254512635379
Precision: 0.9179580674567
Recall   : 0.9225836005497022
F1       : 0.9202650217043636


# Logistic Regression

## Hyperparameter tuning

In [10]:
measures = {"Penalty":[], "C":[], "Solver":[], "Accuracy":[], "Precision":[],"Recall":[], "F1":[]}

done = 0
left = 4*3*3
start = time.time()

for solver in ['newton-cg', 'lbfgs', 'sag', 'saga']:
    for pen in ['l1', 'l2', 'none']:
        for C in [1, 10, 20]:
            try:
                # Print progress
                done, left = progress(done, left, start)
                
                # Train solver
                logr = LogisticRegression(penalty=pen, 
                                          C=C, 
                                          solver=solver,
                                          n_jobs=-1).fit(X_train, y_train)

                # Predict
                y_pred = logr.predict(X_valid)

                measures["Solver"].append(solver)
                measures["Penalty"].append(pen)
                measures["C"].append(C)

                measures["Accuracy"].append(sklearn.metrics.accuracy_score(y_valid, y_pred))
                measures["Precision"].append(sklearn.metrics.precision_score(y_valid, y_pred))
                measures["Recall"].append(sklearn.metrics.recall_score(y_valid, y_pred))
                measures["F1"].append(sklearn.metrics.f1_score(y_valid, y_pred))
            except:
                pass
            
df_logr = pd.DataFrame(measures)
df_logr #.to_csv("logistic_regression.csv")

[################################### ] 97.22% (35/36)
Time passed = 00:14:42


,Penalty,C,Solver,Accuracy,Precision,Recall,F1
0,l2,1,newton-cg,0.894935,0.906030,0.885995,0.895901
1,l2,10,newton-cg,0.916249,0.920831,0.914496,0.917653
2,l2,20,newton-cg,0.918506,0.924107,0.915479,0.919773
3,none,1,newton-cg,0.905216,0.914043,0.898771,0.906343
4,none,10,newton-cg,0.905216,0.914043,0.898771,0.906343
5,none,20,newton-cg,0.905216,0.914043,0.898771,0.906343
6,l2,1,lbfgs,0.894935,0.906030,0.885995,0.895901
7,l2,10,lbfgs,0.915747,0.919506,0.914988,0.917241
8,l2,20,lbfgs,0.915246,0.921510,0.911548,0.916502
9,none,1,lbfgs,0.907472,0.915254,0.902211,0.908686


## Optimal configuration

In [13]:
# Train solver
logr = LogisticRegression(penalty="l2", 
                          C=10, 
                          solver="saga",
                          n_jobs=-1).fit(X_train, y_train)

y_pred = logr.predict(X_test)

print("Accuracy :", sklearn.metrics.accuracy_score(y_test, y_pred))
print("Precision:", sklearn.metrics.precision_score(y_test, y_pred))
print("Recall   :", sklearn.metrics.recall_score(y_test, y_pred))
print("F1       :", sklearn.metrics.f1_score(y_test, y_pred))

Accuracy : 0.9088447653429603
Precision: 0.9048702776513428
Recall   : 0.9106733852496565
F1       : 0.9077625570776255


# Ridge Classification

## Hyperparameter tuning

In [11]:
measures = {"Solver":[], "Alpha":[], "Accuracy":[], "Precision":[], "Recall":[], "F1":[]}

done = 0
left = 3*4
start = time.time()

for solver in ['auto', 'sparse_cg', 'sag']:
    for alpha in [0.1, 1, 10, 100]:
        # Print progress
        done, left = progress(done, left, start)
        
        ridge = RidgeClassifier(solver=solver, alpha=alpha).fit(X_train, y_train)

        y_pred = ridge.predict(X_valid)
        
        measures["Solver"].append(solver)
        measures["Alpha"].append(alpha)

        measures["Accuracy"].append(sklearn.metrics.accuracy_score(y_valid, y_pred))
        measures["Precision"].append(sklearn.metrics.precision_score(y_valid, y_pred))
        measures["Recall"].append(sklearn.metrics.recall_score(y_valid, y_pred))
        measures["F1"].append(sklearn.metrics.f1_score(y_valid, y_pred))

df_ridge = pd.DataFrame(measures)
df_ridge #.to_csv("ridge_classification.csv")

[########### ] 91.67% (11/12)
Time passed = 00:01:04


,Solver,Alpha,Accuracy,Precision,Recall,F1
0,auto,0.1,0.918255,0.919078,0.920885,0.919980
1,auto,1.0,0.917252,0.923497,0.913514,0.918478
2,auto,10.0,0.878134,0.892549,0.865356,0.878743
3,auto,100.0,0.812437,0.854545,0.762162,0.805714
4,sparse_cg,0.1,0.918255,0.919078,0.920885,0.919980
5,sparse_cg,1.0,0.917252,0.923497,0.913514,0.918478
6,sparse_cg,10.0,0.878134,0.892549,0.865356,0.878743
7,sparse_cg,100.0,0.812437,0.854545,0.762162,0.805714
8,sag,0.1,0.918004,0.919450,0.919902,0.919676
9,sag,1.0,0.917252,0.923497,0.913514,0.918478


## Optimal configuration

In [14]:
ridge = RidgeClassifier(solver="auto", alpha=0.1).fit(X_train, y_train)

y_pred = ridge.predict(X_test)

print("Accuracy :", sklearn.metrics.accuracy_score(y_test, y_pred))
print("Precision:", sklearn.metrics.precision_score(y_test, y_pred))
print("Recall   :", sklearn.metrics.recall_score(y_test, y_pred))
print("F1       :", sklearn.metrics.f1_score(y_test, y_pred))

Accuracy : 0.9131317689530686
Precision: 0.9086363636363637
Recall   : 0.9157123224919835
F1       : 0.9121606205795119


# Hybrid Machine Learning

In [38]:
# Predict the label for every video with every optimal model
hybrid = equal[["video_id", "conspiracy"]]
# hybrid["ridge"] = ridge.predict(x)
hybrid["svm"] = svm.predict(x)
hybrid["neural_network"] = mlp.predict(x)
# hybrid["log_reg"] = logr.predict(x)
# hybrid["knn"] = knn.predict(x)

hybrid.head()

,video_id,conspiracy,svm,neural_network
0,SECdW8n4ZGw,False,False,False
1,jSXIrZx8lI4,False,False,False
2,s0YjJVyyVjI,False,False,False
3,VMHCvdRdq10,False,False,False
4,KKiR4FWOZ4w,False,False,False


In [48]:
x2 = hybrid[["svm", "neural_network"]].values

In [49]:
X = x2
Y = hybrid['conspiracy'].values.ravel()

# 10% of the data will be the test set
X_trainval, X_test, y_trainval, y_test = sklearn.model_selection.train_test_split(X, Y, test_size=0.1,\
                                                                                  random_state=0)

# of the remaining 90%, 90% will be training set
X_train, X_valid, y_train, y_valid = sklearn.model_selection.train_test_split(X_trainval, y_trainval,\
                                                                              train_size=0.9,\
                                                                              random_state=0)

In [65]:
ridge = RidgeClassifier(solver="auto", alpha=0.1).fit(X_train, y_train)

y_pred = ridge.predict(X_test)

print("Accuracy :", sklearn.metrics.accuracy_score(y_test, y_pred))
print("Precision:", sklearn.metrics.precision_score(y_test, y_pred))
print("Recall   :", sklearn.metrics.recall_score(y_test, y_pred))
print("F1       :", sklearn.metrics.f1_score(y_test, y_pred))

Accuracy : 0.921254512635379
Precision: 0.9179580674567
Recall   : 0.9225836005497022
F1       : 0.9202650217043636


In [66]:
y_pred, hybrid["svm"].values

(array([False,  True,  True, ...,  True, False,  True]),
 array([False, False, False, ...,  True,  True,  True]))